# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
df = pd.read_csv('main_task.csv')
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

# Для начала приведем стобец с идентификатором ресторана в числовой формат и подсчитаем количество ресторанов
# с одинаковым id
number_rest = df.groupby(['Restaurant_id'])['Ranking'].count()
# Добавляем в датасет колонку c общим кол-вом ресторанов c одинаковым рeйтингом:
df['Number_of_id'] = df.apply(
    lambda row: number_rest[row.Restaurant_id], axis=1)

# преобразование признака в категориальные переменные переменные
df['Price_range_isNAN'] = pd.isna(df['Price Range']).astype('uint8')


def price_range(rang):
    if rang == '$':
        return round(1)
    elif rang == '$$ - $$$':
        return round(2)
    elif rang == '$$$$':
        return round(3)
    else:
        return rang


df['Price Range'] = df['Price Range'].apply(lambda x: price_range(x))

df['Price Range'].fillna(2, inplace=True)


# Cuisune Style
# Отсутствующие значения типа кухонь заменили на Regular, и в количетве представленныъ кухонь считаем равными 1,
# так как минимум одна кухня должна быть представлена, либо некоторые виды являются сочетанием нескольких кухонь
df['Cuisine Style'].fillna('Regular', inplace=True)

# Преобразуем строковые значения из Cuisine Style в списки:
df['cuisine_list'] = df['Cuisine Style'].str.findall(
    r'\w+\s*\w*\s*\w*\s*\w*\s*\w*')
df['cuisine_len'] = df['cuisine_list'].apply(lambda x: len(x))
df.drop(['Cuisine Style'], inplace=True, axis=1)
df.drop(['cuisine_list'], inplace=True, axis=1)

# Так как столбцы URL_TA и ID_TA не несут никакой практической информации было принято решение их удалить
df.drop(['URL_TA'], inplace=True, axis=1)
df.drop(['ID_TA'], inplace=True, axis=1)

# Пропуски в признаке Number of Reviews заполним средним количеством отзывов по городам
for city in df['City'].unique():
    df.loc[df.City == city, ['Number of Reviews']] = df['Number of Reviews'].loc[df.City == city].\
        fillna(round(df['Number of Reviews'].loc[df.City == city].mean(), 2))

In [4]:
# Переведем все даты из отзывов в формат datetime и положим в отдельный признак

def review_to_date(review):
    # Задаем паттерн для даты
    pattern = re.compile('\'\d+\/\d+\/\d+\'?')
    # Записываем даты в переменную
    dat = pattern.findall(review)
    # Переводим даты в формат datetime
    if len(dat) >= 1:
        datetime_list = []
        for date in dat:
            date = date[1:-1]
            dt = datetime.strptime(date, '%m/%d/%Y')
            datetime_list.append(dt)
        return datetime_list


df['review_dates'] = df['Reviews'].apply(lambda x: review_to_date(x))

# Находим разницу в днях между датами каждого второго отзыва с предыдущим, 
#если в датасете был только 1 отзыв то возвращаем 0

def tworeview_day(date):
    if date is not None:
        if len(date)>1:
            delta = date[0] - date[1]
            return abs(round(delta.days))
        else:
            return 0

df['tworeview_day'] = df['review_dates'].apply(lambda x: tworeview_day(x))
df.drop(['Reviews', 'review_dates'], inplace=True, axis=1)

# пропуски заполняем нулями, так как нет объективных данных о разнице дней
df['tworeview_day'] = df['tworeview_day'].fillna(0)

Так в стоблце city нет пропусков, нет ошибок в названии городов и представлено всего 31 город, то было принято решение признак city преобразовать в dummy-переменные

In [5]:
city_dummy = pd.get_dummies(df.City)

df = pd.concat([df, city_dummy], axis=1)      
df.drop(['City'], inplace=True, axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Ranking            40000 non-null  float64
 2   Rating             40000 non-null  float64
 3   Price Range        40000 non-null  float64
 4   Number of Reviews  40000 non-null  float64
 5   Number_of_id       40000 non-null  int64  
 6   Price_range_isNAN  40000 non-null  uint8  
 7   cuisine_len        40000 non-null  int64  
 8   tworeview_day      40000 non-null  float64
 9   Amsterdam          40000 non-null  uint8  
 10  Athens             40000 non-null  uint8  
 11  Barcelona          40000 non-null  uint8  
 12  Berlin             40000 non-null  uint8  
 13  Bratislava         40000 non-null  uint8  
 14  Brussels           40000 non-null  uint8  
 15  Budapest           40000 non-null  uint8  
 16  Copenhagen         400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [7]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [8]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [9]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [10]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [11]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [12]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.220019
